In [1]:
# adapted from: http://www.codiply.com/blog/hyperparameter-grid-search-across-multiple-models-in-scikit-learn/

import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV

class EstimatorSelectionHelper:
    
    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
    
    def fit(self, X, y, **grid_kwargs):
        for key in self.keys:
            print('Running GridSearchCV for %s.' % key)
            model = self.models[key]
            params = self.params[key]
            grid_search = GridSearchCV(model, params, **grid_kwargs)
            grid_search.fit(X, y)
            self.grid_searches[key] = grid_search
        print('Done.')
    
    def score_summary(self, sort_by='mean_test_score'):
        frames = []
        for name, grid_search in self.grid_searches.items():
            frame = pd.DataFrame(grid_search.cv_results_)
            frame = frame.filter(regex='^(?!.*param_).*$')
            frame['estimator'] = len(frame)*[name]
            frames.append(frame)
        df = pd.concat(frames)
        
        df = df.sort_values([sort_by], ascending=False)
        df = df.reset_index()
        df = df.drop(['rank_test_score', 'index'], 1)
        
        columns = df.columns.tolist()
        columns.remove('estimator')
        columns = ['estimator']+columns
        df = df[columns]
        return df

In [2]:
from sklearn import datasets

breast_cancer = datasets.load_breast_cancer()
X_cancer = breast_cancer.data
y_cancer = breast_cancer.target

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

models1 = { 
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

params1 = { 
    'ExtraTreesClassifier': { 'n_estimators': [16, 32] },
    'RandomForestClassifier': [
        { 'n_estimators': [16, 32] },
        {'criterion': ['gini', 'entropy'], 'n_estimators': [8, 16]}],
    'AdaBoostClassifier':  { 'n_estimators': [16, 32] },
    'GradientBoostingClassifier': { 'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] }
}

In [3]:
helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(X_cancer, y_cancer, scoring='f1', n_jobs=2)
helper1.score_summary()

Running GridSearchCV for ExtraTreesClassifier.
Running GridSearchCV for RandomForestClassifier.
Running GridSearchCV for AdaBoostClassifier.
Running GridSearchCV for GradientBoostingClassifier.
Done.


/home/wdsrocha/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/wdsrocha/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/wdsrocha/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/wdsrocha/anaconda3/li

,estimator,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,params,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,AdaBoostClassifier,0.076609,0.003971,0.974914,1.000000,{'n_estimators': 32},0.962343,1.000000,0.991667,1.000000,0.970711,1.000000,0.005322,0.000221,0.012343,0.000000
1,RandomForestClassifier,0.024124,0.001928,0.969320,0.998602,"{'criterion': 'gini', 'n_estimators': 16}",0.962963,1.000000,0.974790,0.997904,0.970213,0.997904,0.000918,0.000080,0.004873,0.000988
2,ExtraTreesClassifier,0.041678,0.003372,0.966561,1.000000,{'n_estimators': 32},0.962025,1.000000,0.967213,1.000000,0.970464,1.000000,0.002627,0.000141,0.003474,0.000000
3,ExtraTreesClassifier,0.029869,0.003440,0.966528,1.000000,{'n_estimators': 16},0.946058,1.000000,0.983333,1.000000,0.970213,1.000000,0.006464,0.000923,0.015451,0.000000
4,AdaBoostClassifier,0.042614,0.002837,0.966487,0.998602,{'n_estimators': 16},0.963265,0.997904,0.983333,0.997904,0.952790,1.000000,0.002386,0.000282,0.012671,0.000988
5,GradientBoostingClassifier,0.022116,0.000702,0.965255,1.000000,"{'learning_rate': 1.0, 'n_estimators': 16}",0.950413,1.000000,0.979253,1.000000,0.966102,1.000000,0.000313,0.000016,0.011799,0.000000
6,RandomForestClassifier,0.052860,0.003253,0.964922,0.999298,{'n_estimators': 32},0.950000,0.997895,0.979253,1.000000,0.965517,1.000000,0.002498,0.000006,0.011960,0.000992
7,RandomForestClassifier,0.018434,0.001467,0.964759,0.998599,"{'criterion': 'entropy', 'n_estimators': 8}",0.953975,1.000000,0.974790,0.995798,0.965517,1.000000,0.003298,0.000005,0.008522,0.001981
8,GradientBoostingClassifier,0.032536,0.000738,0.963882,1.000000,"{'learning_rate': 1.0, 'n_estimators': 32}",0.954733,1.000000,0.979253,1.000000,0.957627,1.000000,0.001433,0.000014,0.010947,0.000000
9,RandomForestClassifier,0.015842,0.001476,0.961920,0.998599,"{'criterion': 'gini', 'n_estimators': 8}",0.966667,0.997895,0.971193,1.000000,0.947826,0.997904,0.003316,0.000020,0.010110,0.000990
